# Importing Required Libraries

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# Reading the data and store it into df

In [2]:
# Load the dataset
df = pd.read_csv('../input/tesla-stock-price/Tesla.csv - Tesla.csv.csv')

# Display the first few rows of the dataset
print(df.head())

        Date       Open   High        Low      Close    Volume  Adj Close
0  6/29/2010  19.000000  25.00  17.540001  23.889999  18766300  23.889999
1  6/30/2010  25.790001  30.42  23.299999  23.830000  17187100  23.830000
2   7/1/2010  25.000000  25.92  20.270000  21.959999   8218800  21.959999
3   7/2/2010  23.000000  23.10  18.709999  19.200001   5139800  19.200001
4   7/6/2010  20.000000  20.00  15.830000  16.110001   6866900  16.110001


# Feature Engineering

In [3]:
# Feature Engineering
# Creating additional features like moving averages
df['MA50'] = df['Close'].rolling(window=50).mean()
df['MA200'] = df['Close'].rolling(window=200).mean()

# Data Preprocessing

In [4]:
# Preprocess the data
# Using Close, High, Low, Volume, MA50, MA200 for prediction
df = df[['Close', 'High', 'Low', 'Open', 'Volume', 'MA50', 'MA200']]

In [5]:
# Handle missing values by filling them with the mean
df.fillna(df.mean(), inplace=True)

In [6]:
# Scale the data to the range (0, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

In [7]:
# Split the data into training and testing sets
training_data_len = int(np.ceil(len(scaled_data) * 0.8))

In [8]:
# Create the training dataset
train_data = scaled_data[0:int(training_data_len), :]

In [9]:
# Split the data into x_train and y_train datasets
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i])
    y_train.append(train_data[i, 0])  # Predicting the 'Close' price


In [10]:
# Convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# LSTM MODEL

In [11]:
# Build the LSTM model
model = Sequential()

In [12]:
# Adding the first LSTM layer with dropout
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dropout(0.2))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
# Adding the second LSTM layer
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

In [14]:
# Adding dense layers
model.add(Dense(units=25))
model.add(Dense(units=1))  # Final output layer

In [15]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [16]:
# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=10)


Epoch 1/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - loss: 0.1020
Epoch 2/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.0066
Epoch 3/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.0051
Epoch 4/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.0034
Epoch 5/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.0034
Epoch 6/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.0031
Epoch 7/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.0031
Epoch 8/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.0029
Epoch 9/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - loss: 0.0030
Epoch 10/10
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - loss: 0.0023


In [17]:
# Create the testing dataset
# Create a new array containing scaled values from index training_data_len to the end
test_data = scaled_data[training_data_len - 60:, :]

In [18]:
# Create the x_test and y_test datasets
x_test = []
y_test = df['Close'][training_data_len:].values

for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i])


In [19]:
# Convert x_test to a numpy array
x_test = np.array(x_test)

In [20]:
# Get the model's predicted price values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(np.hstack((predictions, np.zeros((predictions.shape[0], scaled_data.shape[1]-1)))))[:,0]


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step


# Model Results

In [21]:
# Calculate the RMSE
rmse = sqrt(mean_squared_error(y_test, predictions))
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 15.849165682518784


In [22]:
# Plot the data using Plotly
train = df[:training_data_len]
valid = df[training_data_len:]
valid['Predictions'] = predictions

fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train['Close'], mode='lines', name='Train Close Price'))
fig.add_trace(go.Scatter(x=valid.index, y=valid['Close'], mode='lines', name='Actual Close Price'))
fig.add_trace(go.Scatter(x=valid.index, y=valid['Predictions'], mode='lines', name='Predicted Close Price'))
fig.update_layout(title='Stock Price Prediction',
                  xaxis_title='Date',
                  yaxis_title='Close Price USD ($)')
fig.show()

# Print the valid data with predictions
print(valid[['Close', 'Predictions']])

/tmp/ipykernel_17/3172244062.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


           Close  Predictions
1354  212.940002   208.932412
1355  207.190002   209.108355
1356  214.309998   208.631941
1357  214.000000   207.823197
1358  221.070007   207.141767
...          ...          ...
1687  246.169998   232.770714
1688  258.000000   231.967978
1689  255.729996   231.548505
1690  262.049988   231.771788
1691  261.500000   232.587104

[338 rows x 2 columns]
